In [1]:
import sys
import numpy as np
import pandas as pd
import shutil    
import os
sys.path.append('/afs/cern.ch/eng/tracking-tools/modules')
import pymask as pm

In [2]:
command_log_file='log.madx'
stdout_file='stdout.madx'
Madx = pm.Madxp
with open(stdout_file, 'w') as myFile:
    mad = Madx(stdout=myFile,command_log=command_log_file)

# Create thick optics 20 cm, r=2.5

In [4]:
# Starting from optics file 15 cm, r=3.33
filename0 = 'slhc/round/opt_round_150_1500.madx'
# beta squeeze
betas   = np.arange(0.15, 0.21, 0.01)
# beta pre-squeeze, 2.5*20=50 cm
betas_0 = [0.5 for i in betas]

[0.15 0.16 0.17 0.18 0.19 0.2 ]


In [5]:
for kk in range(len(betas)):
    print('Current study: %s/%s' %(kk+1, len(betas)))
    betx_IP1  = betas[kk]
    betx0_IP1 = betas_0[kk]
    betx_IP5  = betas[kk]
    betx0_IP5 = betas_0[kk]
    bety_IP1  = betas[kk]
    bety0_IP1 = betas_0[kk]
    bety_IP5  = betas[kk]
    bety0_IP5 = betas_0[kk]

    madx_to_read = filename0
    ################## Load initial optics ################## 
    mad.input('''
    Option,  warn,info;
    System,"rm -rf temp";
    System,"mkdir temp";

    system,"ln -fns /afs/cern.ch/eng/lhc/optics/HLLHCV1.5 slhc";
    system,"ln -fns /afs/cern.ch/eng/lhc/optics/runIII lhc";

    call,file="lhc/lhc.seq";
    call,file="slhc/toolkit/macro.madx"; 
    call,file="slhc/hllhc_sequence.madx"; 
    !call,file="slhc/ramp/opt_ramp_1000_1500_thin.madx";
    call,file="%s";
    !exec,myslice;

    exec,mk_beam(7000);
    exec,check_ip(b1); exec,check_ip(b2); 

    use, sequence=lhcb1;
    select, flag=twiss, clear;
    twiss,chrom,file="last_twiss.tfs";

    ''' %(madx_to_read))
    ################## Change betas and betas0 ##################     
    mad.globals.betx_IP1 =betx_IP1
    mad.globals.betx0_IP1=betx0_IP1
    mad.globals.betx_IP5 =betx_IP5
    mad.globals.betx0_IP5 =betx0_IP5

    mad.globals.bety_IP1 =bety_IP1
    mad.globals.bety0_IP1=bety0_IP1
    mad.globals.bety_IP5 =bety_IP5
    mad.globals.bety0_IP5 =bety0_IP5

    if kk==len(betas)-1:
        print("Requested betas: ", mad.globals.betx_IP1, mad.globals.betx0_IP1, mad.globals.betx_IP5, mad.globals.betx0_IP5)

    
    ################## Run rematch_hllhc script up to W rematch ################## 
    mad.input('''

    exec, round_phases;

    exec,rematch_arc(1,2,12);
    exec,rematch_arc(2,3,23);
    exec,rematch_arc(3,4,34);
    exec,rematch_arc(4,5,45);
    exec,rematch_arc(5,6,56);
    exec,rematch_arc(6,7,67);
    exec,rematch_arc(7,8,78);
    exec,rematch_arc(8,1,81);

    scxir1=betx_IP1/betx0_IP1;
    scyir1=bety_IP1/bety0_IP1;
    scxir5=betx_IP5/betx0_IP5;
    scyir5=bety_IP5/bety0_IP5;

    value,scxir1,scyir1,scxir5,scyir5;

    exec,selectIR15(5,45,56,b1);
    exec,selectIR15(5,45,56,b2);
    exec,selectIR15(1,81,12,b1);
    exec,selectIR15(1,81,12,b2);

    on_holdselect=1; jac_calls=   15;jac_tol=1e-20; jac_bisec=3;

    if (sc79==0){sc79=0.999;};
    if (sch==0) {sch =0.99; };
    if (imb==0) {imb=1.50;};
    if (scl==0) {scl=0.06;};
    grad=132.6; bmaxds=500;
    betxip5b1=betx0_IP5; betyip5b1=bety0_IP5;
    betxip5b2=betx0_IP5; betyip5b2=bety0_IP5;
    betxip1b1=betx0_IP1; betyip1b1=bety0_IP1;
    betxip1b2=betx0_IP1; betyip1b2=bety0_IP1;

    match_on_triplet=0; call,file="slhc/toolkit/rematch_ir15b12.madx";
    call,file="slhc/toolkit/rematch_ir15b12.madx";

    betxip5b1=betx_IP5; betyip5b1=bety_IP5;
    betxip5b2=betx_IP5; betyip5b2=bety_IP5;
    betxip1b1=betx_IP1; betyip1b1=bety_IP1;
    betxip1b2=betx_IP1; betyip1b2=bety_IP1;

    exec,select(3,23,34,b1);
    exec,select(3,23,34,b2);
    exec,select(7,67,78,b1);
    exec,select(7,67,78,b2);

    call,file="slhc/toolkit/rematch_ir3b1.madx";
    call,file="slhc/toolkit/rematch_ir3b2.madx";
    call,file="slhc/toolkit/rematch_ir7b1.madx";
    call,file="slhc/toolkit/rematch_ir7b2.madx";


    exec,selectIRAUX(7,8,1,2,3,b1,scxir1,scyir1,betx0_ip1,bety0_ip1);
    exec,selectIRAUX(7,8,1,2,3,b2,scxir1,scyir1,betx0_ip1,bety0_ip1);
    exec,selectIRAUX(3,4,5,6,7,b1,scxir5,scyir5,betx0_ip5,bety0_ip5);
    exec,selectIRAUX(3,4,5,6,7,b2,scxir5,scyir5,betx0_ip5,bety0_ip5);

    call,file="slhc/toolkit/rematch_ir2b12.madx";
    call,file="slhc/toolkit/rematch_ir8b12.madx";
    call,file="slhc/toolkit/rematch_ir4b1.madx";
    call,file="slhc/toolkit/rematch_ir4b2.madx";
    jac_calls=   15;
    !dxip6b1=0;dpxip6b1=0; dxip6b2=0;dpxip6b2=0;
    !call,file=rematch_ir6b1m.madx;
    !call,file=rematch_ir6b2m.madx;
    !dxip6b1=0;dpxip6b1=0; dxip6b2=0;dpxip6b2=0;
    nomatch_dx=1; nomatch_dpx=1;
    call,file="slhc/toolkit/rematch_ir6b1.madx";
    call,file="slhc/toolkit/rematch_ir6b2.madx";
    nomatch_dx=0; nomatch_dpx=0;
    call,file="slhc/toolkit/rematch_ir6b1.madx";
    call,file="slhc/toolkit/rematch_ir6b2.madx";
    value,tarir2b1,tarir4b1,tarir6b1,tarir8b1;
    value,tarir2b2,tarir4b2,tarir6b2,tarir8b2;
    value,tarir3b1,tarir7b1,tarir3b2,tarir7b2;
    value,tarir1b1,tarir5b1,tarir1b2,tarir5b2;


    tarsqueeze=tarir2b1+tarir4b1+tarir6b1+tarir8b1+
               tarir2b2+tarir4b2+tarir6b2+tarir8b2+
               tarir3b1+tarir7b1+tarir3b2+tarir7b2+
               tarir1b1+tarir5b1+tarir1b2+tarir5b2;

    exec,check_ip(b1); exec,check_ip(b2);
    value,tarsqueeze;

    if (tarsqueeze>1e-15 || match_optics_only==1){return;};


    call,file="slhc/toolkit/rematch_xing_ir15.madx";
    call,file="slhc/toolkit/rematch_xing_ir28.madx";

    value,tar_xing_ir15,tar_xing_ir28;

    ksf=0.06;ksd=-0.099;
    exec,set_sext_all(ksf,ksd,ksf,ksd); k2max=0.38;
    ''')
    
    ################## Save intermediate optics in optics dir##################
    filename = 'opt_round_%s_1500_set_sext_all' %int(mad.globals.betx_IP1*1000)
    mad.input('''
    exec,save_optics_hllhc("%s.madx");
    ''' %filename)
    if not os.path.exists('optics'):
        os.makedirs('optics')
    shutil.move("%s.madx" %filename, "optics/%s.madx" %filename)
    filename0 = "optics/%s.madx" %filename 
    
    if kk==len(betas)-1:
        print("Final betas: ", mad.globals.betx_IP1, mad.globals.betx0_IP1, mad.globals.betx_IP5, mad.globals.betx0_IP5)


Current study: 1/6
Current study: 2/6
Current study: 3/6
Current study: 4/6
Current study: 5/6
Current study: 6/6
Requested betas:  0.20000000000000004 0.5 0.20000000000000004 0.5
Final betas:  0.20000000000000004 0.5000000000007427 0.20000000000000004 0.4999999999998181


In [6]:
################## Rematch W, disp & crabs ##################
filename = 'opt_round_7TeV_%s_1500' %int(round(mad.globals.betx_IP1,2)*1000)

mad.input('''
if (match_w2==0){
call,file="slhc/toolkit/rematch_w.madx";
call,file="slhc/toolkit/rematch_disp.madx";
};
if (match_w2==2){
call,file="slhc/toolkit/rematch_w2.madx";
call,file="slhc/toolkit/rematch_disp2.madx";
};

call,file="slhc/toolkit/rematch_crabs.madx";
call,file="slhc/toolkit/mk_arc_trims.madx";

value,tarsqueeze, tar_xing_ir15, tar_xing_ir28, tar_on_disp;

exec,save_optics_hllhc("%s.madx");
use, sequence=lhcb1;
select, flag=twiss, clear;
twiss,chrom,file="last_twiss.tfs";

'''%filename)

True

In [7]:
twiss_final = mad.get_twiss_df(table_name='twiss')
summ_final = (mad.table['summ'].dframe())

print('Betas:')
print(twiss_final.loc['ip1:1'].betx, twiss_final.loc['ip2:1'].betx, twiss_final.loc['ip5:1'].betx, twiss_final.loc['ip8:1'].betx)
print('Final tunes and chroma::')
print(summ_final['q1'].values[0], summ_final['q2'].values[0], summ_final['dq1'].values[0], summ_final['dq2'].values[0])
print('tarsqueeze: ', mad.globals.tarsqueeze)

Betas:
0.19999999998976287 10.000000000633657 0.19999999998731588 1.4999999998835791
Final tunes and chroma::
62.3100000000181 60.320000000010346 1.9994823148294927 2.00008621931147
tarsqueeze:  2.7024115234459403e-20


# Create thin optics 20 cm, r=2.5

In [3]:
command_log_file='log.madx'
stdout_file='stdout.madx'
Madx = pm.Madxp
with open(stdout_file, 'w') as myFile:
    mad = Madx(stdout=myFile,command_log=command_log_file)

In [4]:
filename = "opt_round_7TeV_200_1500"
mad.input('''
    Option,  warn,info;
    System,"rm -rf temp";
    System,"mkdir temp";

    system,"ln -fns /afs/cern.ch/eng/lhc/optics/HLLHCV1.5 slhc";
    system,"ln -fns /afs/cern.ch/eng/lhc/optics/runIII lhc";

    call,file="lhc/lhc.seq";
    call,file="slhc/toolkit/macro.madx"; 
    call,file="slhc/hllhc_sequence.madx"; 
    call,file="%s.madx";
    exec,myslice;

    exec,mk_beam(7000);
    !exec,check_ip(b1); exec,check_ip(b2); 
    call,file="slhc/toolkit/rematch_hllhc.madx"; 
    exec,save_optics_hllhc("%s_thin.madx");

'''%(filename, filename))

True

# No MS.10 thick optics

In [2]:
command_log_file='log.madx'
stdout_file='stdout.madx'
Madx = pm.Madxp
with open(stdout_file, 'w') as myFile:
    mad = Madx(stdout=myFile,command_log=command_log_file)

In [3]:
filename = "opt_round_7TeV_200_1500"
mad.input('''
Option,  warn,info;
System,"rm -rf temp";
System,"mkdir temp";

system,"ln -fns /afs/cern.ch/eng/lhc/optics/HLLHCV1.5 slhc";
system,"ln -fns /afs/cern.ch/eng/lhc/optics/runIII lhc";

call,file="lhc/lhc.seq";
call,file="slhc/toolkit/macro.madx"; 
call,file="slhc/hllhc_sequence.madx"; 
call,file="%s.madx";


exec,mk_beam(7000);
exec,check_ip(b1); exec,check_ip(b2); 
!call,file="slhc/toolkit/rematch_hllhc.madx"; 

use, sequence=lhcb1;
select, flag=twiss, clear;
twiss,chrom,file="initial_twiss.tfs";

'''%filename)

twiss_initial = mad.get_twiss_df(table_name='twiss')
summ_initial  = mad.table.summ.dframe()

In [4]:
filename = "opt_round_NoMS10_7TeV_200_1500"

mad.input('''
exec, DISABLE_SEXT(MS.10);
ksf=0.06;ksd=-0.099;
exec,set_sext_all(ksf,ksd,ksf,ksd); k2max=0.38;

if (match_w2==0){
call,file="slhc/toolkit/rematch_w.madx";
call,file="slhc/toolkit/rematch_disp.madx";
};
if (match_w2==2){
call,file="slhc/toolkit/rematch_w2.madx";
call,file="slhc/toolkit/rematch_disp2.madx";
};

call,file="slhc/toolkit/rematch_crabs.madx";
call,file="slhc/toolkit/mk_arc_trims.madx";

value,tarsqueeze, tar_xing_ir15, tar_xing_ir28, tar_on_disp;

exec,save_optics_hllhc("%s.madx");

use, sequence=lhcb1;
select, flag=twiss, clear;
twiss,chrom,file="last_twiss.tfs";

'''%filename)

twiss_final = mad.get_twiss_df(table_name='twiss')
summ_final  = mad.table.summ.dframe()


In [5]:

# beta in IP1,2,5,8
print('Betas:')
print(twiss_final.loc['ip1:1'].betx, twiss_final.loc['ip2:1'].betx, twiss_final.loc['ip5:1'].betx, twiss_final.loc['ip8:1'].betx)

# tunes & chromaticity
print('Initial tunes and chroma:')
print(summ_initial['q1'].values[0], summ_initial['q2'].values[0], summ_initial['dq1'].values[0], summ_initial['dq2'].values[0])
print('Final tunes and chroma::')
print(summ_final['q1'].values[0], summ_final['q2'].values[0], summ_final['dq1'].values[0], summ_final['dq2'].values[0])


# MS.10 k2l
print('Initial k2l MS.10:')
print(twiss_initial[twiss_initial.index.str.contains('ms.10')]['k2l'])
print('Final k2l MS.10:')
print(twiss_final[twiss_final.index.str.contains('ms.10')]['k2l'])

Betas:
0.19999999998981188 10.00000000063152 0.19999999998732493 1.499999999883938
Initial tunes and chroma:
62.3100000000037 60.320000000000434 1.9994826487845785 2.0000871998604453
Final tunes and chroma::
62.31000000001794 60.32000000001016 1.9987614123806452 2.0009106478369176
Initial k2l MS.10:

ms.10r1.b1:1   -0.134393
ms.10l5.b1:1    0.080561
ms.10r5.b1:1   -0.134093
ms.10l1.b1:1    0.100547
Name: k2l, dtype: float64
Final k2l MS.10:

ms.10l5.b1:1    0.0
ms.10r5.b1:1    0.0
ms.10l1.b1:1    0.0
ms.10r1.b1:1    0.0
Name: k2l, dtype: float64


# No MS.10 thin optics

In [3]:
command_log_file='log.madx'
stdout_file='stdout.madx'
Madx = pm.Madxp
with open(stdout_file, 'w') as myFile:
    mad = Madx(stdout=myFile,command_log=command_log_file)

In [4]:
filename = "opt_round_NoMS10_7TeV_200_1500"
mad.input('''
    Option,  warn,info;
    System,"rm -rf temp";
    System,"mkdir temp";

    system,"ln -fns /afs/cern.ch/eng/lhc/optics/HLLHCV1.5 slhc";
    system,"ln -fns /afs/cern.ch/eng/lhc/optics/runIII lhc";

    call,file="lhc/lhc.seq";
    call,file="slhc/toolkit/macro.madx"; 
    
    call,file="slhc/hllhc_sequence.madx"; 
    exec, DISABLE_SEXT(MS.10);
    
    call,file="%s.madx";
    exec,myslice;

    exec,mk_beam(7000);
    !exec,check_ip(b1); exec,check_ip(b2); 
    call,file="slhc/toolkit/rematch_hllhc.madx"; 
    exec,save_optics_hllhc("%s_thin.madx");

'''%(filename, filename))

True

# Sanity check thin optics no MS.10

In [5]:
command_log_file='log2.madx'
stdout_file='stdout2.madx'
Madx = pm.Madxp
with open(stdout_file, 'w') as myFile:
    mad2 = Madx(stdout=myFile,command_log=command_log_file)

In [6]:
filename = "opt_round_NoMS10_7TeV_200_1500_thin"
mad2.input('''
Option,  warn,info;
System,"rm -rf temp";
System,"mkdir temp";

system,"ln -fns /afs/cern.ch/eng/lhc/optics/HLLHCV1.5 slhc";
system,"ln -fns /afs/cern.ch/eng/lhc/optics/runIII lhc";

call,file="lhc/lhc.seq";
call,file="slhc/toolkit/macro.madx"; 
call,file="slhc/hllhc_sequence.madx"; 

exec, DISABLE_SEXT(MS.10);

call,file="%s.madx";

exec,myslice;

exec,mk_beam(7000);
exec,check_ip(b1); exec,check_ip(b2); 
!call,file="slhc/toolkit/rematch_hllhc.madx"; 

use, sequence=lhcb1;
select, flag=twiss, clear;
twiss,chrom,file="initial_twiss.tfs";

'''%filename)

twiss_initial = mad2.get_twiss_df(table_name='twiss')
summ_initial  = mad2.table.summ.dframe()

In [7]:
print('Betas:')
print(twiss_initial.loc['ip1:1'].betx, twiss_initial.loc['ip2:1'].betx, twiss_initial.loc['ip5:1'].betx, twiss_initial.loc['ip8:1'].betx)

# tunes & chromaticity
print('Initial tunes and chroma:')
print(summ_initial['q1'].values[0], summ_initial['q2'].values[0], summ_initial['dq1'].values[0], summ_initial['dq2'].values[0])


# MS.10 k2l
print('Initial k2l MS.10:')
print(twiss_initial[twiss_initial.index.str.contains('ms.10')]['k2l'])

Betas:
0.20000000000006365 9.99999999999534 0.20000000000012588 1.5000000000015532
Initial tunes and chroma:
62.3100004654774 60.32000046102331 1.980000646994995 1.9993189468436867
Initial k2l MS.10:

ms.10r1.b1:1    0.0
ms.10l5.b1:1    0.0
ms.10r5.b1:1    0.0
ms.10l1.b1:1    0.0
Name: k2l, dtype: float64
